# Postage Stamps

This notebook creates postage stamps for subtracted images generates by the Diference Image Analysis DIA pipeline. For instructions on running the pipeline, see this project's ReadMe.


## Setup

You will need to specify the directory of the differenced image you wish to process. This directory should contain (or contain subdirectories with) both the differnece image itself, along with the object cataloge for that image (which is autoimatically generated byt the DIA pipeline).

In [90]:
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.table import Table
from astropy.wcs import WCS
from pathlib import Path

# Input DIA data
diff_im_dir = '/global/u1/d/djp81/test_imdiff/deepDiff/'

# Output directory for postave stamps
postage_output_dir = './stamps'


## Understanding and Parsing DIA Data

Subtraction results from the DIA pipeline are saved as pairs of `diffexp` and `diaSrc` files. The `diffexp` files contain the subracted image data while the `diaSrc` files act as a catlog of objects found in the subtracted image. We begin by collecting the paths of any DIA output files in the directory `diff_im_dir` specified above.


In [45]:
def get_diff_file_paths(dia_dir):
    """Return lists of all diffexp and diaSrc file paths under ``dia_dir``
    
    Args:
        dia_dir (str): Path of DIA pipeline outputs
        
    Returns:
        A list of all diffexp paths as Path objects
        A list of all diaSrc paths as Path objects
    """

    diff_files, dia_files = [], []
    for filename in Path(dia_dir).glob('**/diffexp*.fits'):
        diff_files.append(filename)
        dia_files.append(Path(str(filename).replace('diffexp', 'diaSrc')))
        
    return diff_files, dia_files


diffexp_paths, diasrc_paths = get_diff_file_paths(diff_im_dir)

print('diffexp paths:')
print(diffexp_paths)

print('\ndiaSrc paths:')
print(diasrc_paths, '\n')


diffexp paths:
[PosixPath('/global/u1/d/djp81/test_imdiff/deepDiff/v00431306-fu/R10/diffexp_00431306-u-R10-S01-det028.fits')]

diaSrc paths:
[PosixPath('/global/u1/d/djp81/test_imdiff/deepDiff/v00431306-fu/R10/diaSrc_00431306-u-R10-S01-det028.fits')] 



**Question / Todo:** Where can I go to find a description / data model for the diaSrc files? Put a summary here (relevent explenation only).

Lets pause to familiarize ourselves with a summary of the objects found in the first subtracted image.


In [22]:
diaSrc_hdu_list = fits.open(diasrc_paths[0])
diaSrc_data = diaSrc_hdu_list[1].data
diaSrc_table = Table(np.array(diaSrc_data))

print('Number of diaSrc objects:', len(diaSrc_table))
print('The first ten entries:\n')
diaSrc_table[:10]


Number of diaSrc objects: 2202
The first ten entries:



flags [10],id,coord_ra,coord_dec,parent,base_NaiveCentroid_x,base_NaiveCentroid_y,base_PeakCentroid_x,base_PeakCentroid_y,base_SdssCentroid_x,base_SdssCentroid_y,base_SdssCentroid_xErr,base_SdssCentroid_yErr,ip_diffim_NaiveDipoleCentroid_x,ip_diffim_NaiveDipoleCentroid_y,ip_diffim_NaiveDipoleCentroid_xErr,ip_diffim_NaiveDipoleCentroid_yErr,ip_diffim_NaiveDipoleCentroid_pos_x,ip_diffim_NaiveDipoleCentroid_pos_y,ip_diffim_NaiveDipoleCentroid_pos_xErr,ip_diffim_NaiveDipoleCentroid_pos_yErr,ip_diffim_NaiveDipoleCentroid_neg_x,ip_diffim_NaiveDipoleCentroid_neg_y,ip_diffim_NaiveDipoleCentroid_neg_xErr,ip_diffim_NaiveDipoleCentroid_neg_yErr,base_SdssShape_xx,base_SdssShape_yy,base_SdssShape_xy,base_SdssShape_xxErr,base_SdssShape_yyErr,base_SdssShape_xyErr,base_SdssShape_x,base_SdssShape_y,base_SdssShape_instFlux,base_SdssShape_instFluxErr,base_SdssShape_psf_xx,base_SdssShape_psf_yy,base_SdssShape_psf_xy,base_SdssShape_instFlux_xx_Cov,base_SdssShape_instFlux_yy_Cov,base_SdssShape_instFlux_xy_Cov,base_CircularApertureFlux_3_0_instFlux,base_CircularApertureFlux_3_0_instFluxErr,base_CircularApertureFlux_4_5_instFlux,base_CircularApertureFlux_4_5_instFluxErr,base_CircularApertureFlux_6_0_instFlux,base_CircularApertureFlux_6_0_instFluxErr,base_CircularApertureFlux_9_0_instFlux,base_CircularApertureFlux_9_0_instFluxErr,base_CircularApertureFlux_12_0_instFlux,base_CircularApertureFlux_12_0_instFluxErr,base_CircularApertureFlux_17_0_instFlux,base_CircularApertureFlux_17_0_instFluxErr,base_CircularApertureFlux_25_0_instFlux,base_CircularApertureFlux_25_0_instFluxErr,base_CircularApertureFlux_35_0_instFlux,base_CircularApertureFlux_35_0_instFluxErr,base_CircularApertureFlux_50_0_instFlux,base_CircularApertureFlux_50_0_instFluxErr,base_CircularApertureFlux_70_0_instFlux,base_CircularApertureFlux_70_0_instFluxErr,base_GaussianFlux_instFlux,base_GaussianFlux_instFluxErr,base_PeakLikelihoodFlux_instFlux,base_PeakLikelihoodFlux_instFluxErr,base_PsfFlux_instFlux,base_PsfFlux_instFluxErr,base_PsfFlux_area,ip_diffim_NaiveDipoleFlux_pos_instFlux,ip_diffim_NaiveDipoleFlux_pos_instFluxErr,ip_diffim_NaiveDipoleFlux_neg_instFlux,ip_diffim_NaiveDipoleFlux_neg_instFluxErr,ip_diffim_NaiveDipoleFlux_npos,ip_diffim_NaiveDipoleFlux_nneg,ip_diffim_PsfDipoleFlux_pos_instFlux,ip_diffim_PsfDipoleFlux_pos_instFluxErr,ip_diffim_PsfDipoleFlux_neg_instFlux,ip_diffim_PsfDipoleFlux_neg_instFluxErr,ip_diffim_PsfDipoleFlux_chi2dof,ip_diffim_PsfDipoleFlux_pos_centroid_x,ip_diffim_PsfDipoleFlux_pos_centroid_y,ip_diffim_PsfDipoleFlux_pos_centroid_xErr,ip_diffim_PsfDipoleFlux_pos_centroid_yErr,ip_diffim_PsfDipoleFlux_neg_centroid_x,ip_diffim_PsfDipoleFlux_neg_centroid_y,ip_diffim_PsfDipoleFlux_neg_centroid_xErr,ip_diffim_PsfDipoleFlux_neg_centroid_yErr,ip_diffim_PsfDipoleFlux_centroid_x,ip_diffim_PsfDipoleFlux_centroid_y,ip_diffim_PsfDipoleFlux_centroid_xErr,ip_diffim_PsfDipoleFlux_centroid_yErr,ip_diffim_ClassificationDipole_value,ip_diffim_DipoleFit_pos_instFlux,ip_diffim_DipoleFit_pos_instFluxErr,ip_diffim_DipoleFit_pos_centroid_x,ip_diffim_DipoleFit_pos_centroid_y,ip_diffim_DipoleFit_neg_instFlux,ip_diffim_DipoleFit_neg_instFluxErr,ip_diffim_DipoleFit_neg_centroid_x,ip_diffim_DipoleFit_neg_centroid_y,ip_diffim_DipoleFit_centroid_x,ip_diffim_DipoleFit_centroid_y,ip_diffim_DipoleFit_instFlux,ip_diffim_DipoleFit_orientation,ip_diffim_DipoleFit_separation,ip_diffim_DipoleFit_chi2dof,ip_diffim_DipoleFit_signalToNoise,totFlux,totFluxErr,footprint
uint8,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float64,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float6

In [38]:
diffexp_hdu_list = fits.open(diffexp_paths[0])
# Todo: Plot some images


()
(4000, 4072)
(4000, 4072)
(4000, 4072)
(50,)
(46,)
(23,)
(23,)
(1,)
(1,)
(1,)


## Generating Postage Stamps

In [92]:
def cutout_image_object(
        diffexp_hdu_list, position, size, out_path, overwrite=False):
    """Save a diffexp cutout to file
    
    Args:
        diffexp_hdu_list (HDUList): HDU list of dia difference image data
        position        (SkyCoord): Center position of cutout
        size     (int, Tuple[int]): Size of cutout in pixels
        out_path             (str): Output fits path of cutout image
        overwrite           (bool): Overwrite existing files (Default: False) 
    """

    out_data = fits.HDUList(hdus=[diffexp_hdu_list[0]])
    for hdu in diffexp_hdu_list[1:4]:
        cutout = Cutout2D(
            hdu.data,
            position=position,
            size=size,
            wcs=WCS(hdu.header))

        new_hdu = fits.hdu.FitsHDU(
            data=cutout.data,
            header=cutout.wcs.to_header())

        out_data.append(new_hdu)

    # Enforce .fits file extension
    out_path = str(Path(out_path).with_suffix('.fits'))
    out_data.writeto(out_path, output_verify='ignore', overwrite=overwrite)

def create_postage_stamps(
        diffexp_path, diasrc_path, size, out_dir, overwrite=False):
    """Create postage stamps for all DIA sources in a difference image
    
    Args:
        diffexp_path     (str): Path of a DIA pipeline diffexp image 
        diasrc_path      (str): Path of a DIA pipeline diasrc catalog
        size (int, Tuple[int]): Size of cutout in pixels
        out_dir          (str): Output directory of postage stamps
        overwrite       (bool): Overwrite existing files (Default: False)
    """

    # Create output directories
    sub_dir = Path(out_dir) / str(Path(diffexp_path).stem).lstrip('diff_exp_')
    sub_dir.mkdir(exist_ok=True, parents=True)

    # Create postage stamps
    difexp_hdu_list = fits.open(diffexp_path)
    diaSrc_hdu_list = fits.open(diasrc_path)
    for dia_src in diaSrc_hdu_list[1].data:
        out_path = sub_dir / f'{dia_src["id"]}.fits'
        position = SkyCoord(ra=dia_src['coord_ra'], dec=dia_src['coord_dec'], unit='rad')
        cutout_image_object(difexp_hdu_list, position, size, out_path, overwrite)


In [ ]:
create_postage_stamps(
    diffexp_paths[0], diasrc_paths[0], 20, postage_output_dir)
